In [8]:
import json
import psycopg2

# Kết nối đến PostgreSQL
DB_NAME = "postgres"
DB_USER = "postgres"
DB_PASSWORD = "minh0985362932"
DB_HOST = "localhost"
DB_PORT = "5432"

try:
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    cursor = conn.cursor()

    # Đọc file JSON
    with open("Data/chunked_df.json", "r", encoding="utf-8") as file:
        data = json.load(file)

    # Chèn dữ liệu vào bảng PostgreSQL
    for entry in data:
        cursor.execute("""
            INSERT INTO articles_about_health_care (id, id_baiviet, content) 
            VALUES (%s, %s, %s)
            ON CONFLICT (id) DO NOTHING;  -- Tránh lỗi trùng ID
        """, (entry["id"], entry["id_baiviet"], entry["content"]))

    # Lưu thay đổi
    conn.commit()
    print("Dữ liệu đã được ghi vào PostgreSQL!")

except Exception as e:
    print("Lỗi:", e)

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()


Dữ liệu đã được ghi vào PostgreSQL!


In [10]:
try:
    # Kết nối đến PostgreSQL
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    cursor = conn.cursor()

    # Đọc file JSON
    with open("Data/classification_query.json", "r", encoding="utf-8") as file:
        data = json.load(file)

    # Ghi dữ liệu vào PostgreSQL
    for entry in data:
        cursor.execute("""
            INSERT INTO embeddings_query (texts, labels, embeddings)
            VALUES (%s, %s, %s);
        """, (entry["texts"], entry["labels"], json.dumps(entry["embeddings"])))  # Chuyển embeddings thành JSON

    # Lưu thay đổi
    conn.commit()
    print("Dữ liệu đã được ghi vào PostgreSQL!")

except Exception as e:
    print("Lỗi:", e)

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

Dữ liệu đã được ghi vào PostgreSQL!


In [1]:
import psycopg2
import json
from collections import Counter
from rank_bm25 import BM25Okapi
from pyvi import ViTokenizer
from tqdm import tqdm  # Import tqdm để hiển thị progress bar

# Đọc danh sách stopwords từ file
def load_stopwords(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        stopwords = set(f.read().splitlines())
    return stopwords

stopwords = load_stopwords("vietnamese-stopwords-dash.txt")  

# Kết nối PostgreSQL
DB_NAME = "postgres"
DB_USER = "postgres"
DB_PASSWORD = "minh0985362932"
DB_HOST = "localhost"
DB_PORT = "5432"

conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cursor = conn.cursor()

# Lấy bài viết từ database
cursor.execute("SELECT id, content FROM articles_about_health_care")
rows = cursor.fetchall()

if not rows:
    print("❌ Không có bài viết nào trong database!")
    cursor.close()
    conn.close()
    exit()

# Xử lý TF cho cả lowercase & uppercase
tf_corpus_lower = []
tf_corpus_upper = []
id_list = []

print("🔄 Đang xử lý dữ liệu...")
for id_baiviet, content in tqdm(rows, desc="Tokenizing & TF Processing"):
    if not content or content.strip() == "":
        print(f"⚠️ Bỏ qua ID {id_baiviet} (nội dung trống)")
        continue

    # Tokenize với lowercase
    tokenized_text_lower = ViTokenizer.tokenize(content.lower()).split()
    tokenized_text_lower = [word for word in tokenized_text_lower if word not in stopwords]
    
    # Tokenize với nguyên gốc
    tokenized_text_upper = ViTokenizer.tokenize(content).split()
    tokenized_text_upper = [word for word in tokenized_text_upper if word not in stopwords]

    if not tokenized_text_lower or not tokenized_text_upper:
        print(f"⚠️ Bỏ qua ID {id_baiviet} (không còn từ hợp lệ)")
        continue

    # Tính tần suất từ (TF)
    tf_lower = dict(Counter(tokenized_text_lower))
    tf_upper = dict(Counter(tokenized_text_upper))

    tf_corpus_lower.append(tf_lower)
    tf_corpus_upper.append(tf_upper)
    id_list.append(id_baiviet)

if not tf_corpus_lower or not tf_corpus_upper:
    print("❌ Không có dữ liệu hợp lệ để lưu TF!")
    cursor.close()
    conn.close()
    exit()

# Lưu TF vào database với tqdm
data_to_insert = []
for idx, id_baiviet in tqdm(enumerate(id_list), total=len(id_list), desc="Saving to DB"):
    tf_json_lower = json.dumps(tf_corpus_lower[idx])
    tf_json_upper = json.dumps(tf_corpus_upper[idx])
    data_to_insert.append((id_baiviet, tf_json_lower, tf_json_upper))

cursor.executemany("""
    INSERT INTO word_frequency_health_paragraphs (id, tf_lower, tf_upper)
    VALUES (%s, %s, %s)
    ON CONFLICT (id) DO UPDATE 
    SET tf_lower = EXCLUDED.tf_lower,
        tf_upper = EXCLUDED.tf_upper;
""", data_to_insert)

conn.commit()
print("✅ Tần suất từ (TF) đã được lưu vào PostgreSQL!")

# Đóng kết nối
cursor.close()
conn.close()


🔄 Đang xử lý dữ liệu...


Tokenizing & TF Processing:   7%|▋         | 20957/317023 [03:19<41:23, 119.19it/s] 

⚠️ Bỏ qua ID longchau_baiviet_article_data18110_0 (nội dung trống)


Tokenizing & TF Processing:  20%|██        | 63458/317023 [10:07<38:57, 108.46it/s] 

⚠️ Bỏ qua ID longchau_baiviet_article_data14035_6 (nội dung trống)


...

⚠️ Bỏ qua ID longchau_baiviet_article_data28303_6 (nội dung trống)


Tokenizing & TF Processing:  79%|███████▉  | 251996/317023 [39:58<09:25, 115.06it/s]

⚠️ Bỏ qua ID vinmec_article_data26316_1 (nội dung trống)


Saving to DB: 100%|██████████| 317017/317017 [00:11<00:00, 28013.92it/s]


✅ Tần suất từ (TF) đã được lưu vào PostgreSQL!


In [19]:
import psycopg2
import json
from collections import Counter
from rank_bm25 import BM25Okapi
from pyvi import ViTokenizer
from tqdm import tqdm  # Import tqdm để hiển thị progress bar
from dotenv import load_dotenv
import os

# Load API key từ .env
load_dotenv()
api_key = os.getenv("HUGGINGFACE_API_KEY")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")

In [20]:
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cursor = conn.cursor()
cursor.execute("SELECT id, tf_lower FROM word_frequency_health_paragraphs")
rows = cursor.fetchall()

tokenized_corpus = []
id_list = []

for id_baiviet, tf_json in rows:
    if isinstance(tf_json, dict):  # Nếu đã là dict, không cần json.loads
        tf = tf_json
    elif isinstance(tf_json, str):  # Nếu là chuỗi, parse JSON
        tf = json.loads(tf_json)
    elif tf_json is None:  # Nếu là NULL trong PostgreSQL
        tf = {}
    else:
        raise TypeError(f"❌ Dữ liệu không hợp lệ cho ID {id_baiviet}: {type(tf_json)}")
    
    tokenized_corpus.append(list(tf.keys()))
    id_list.append(id_baiviet)

bm25 = BM25Okapi(tokenized_corpus)  # Tạo BM25 với dữ liệu mới
cursor.close()

In [31]:
from pyvi import ViTokenizer

query = "đậu đen có tác dụng gì cho sức khỏe"  # Câu truy vấn
tokenized_query = ViTokenizer.tokenize(query.lower()).split()  # Tokenize & lowercase

scores = bm25.get_scores(tokenized_query)  # Tính điểm BM25 cho mỗi văn bản

# Sắp xếp kết quả theo điểm số từ cao đến thấp
sorted_results = sorted(zip(id_list, scores), key=lambda x: x[1], reverse=True)

# In kết quả
for doc_id, score in sorted_results[:10]:  # Lấy top 10 kết quả
    print(f"ID: {doc_id}, Score: {score}")
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cursor = conn.cursor()

top_ids = [doc_id for doc_id, _ in sorted_results[:4]]  # Lấy 3 ID có điểm cao nhất
context = ""
for i,doc_id in enumerate(top_ids):
    cursor.execute("SELECT content FROM articles_about_health_care WHERE id = %s", (doc_id,))
    result = cursor.fetchone()
    
    if result:
        print(f"\n📰 Bài viết liên quan (ID: {doc_id}):")
        text = result[0].replace("\n", " ").replace("  ", " ")
        print(text)
        text = f'''Đoạn {i}: "{text}"\n'''
        context += text
    else:
        print(f"❌ Không tìm thấy bài viết cho ID: {doc_id}")


cursor.close()
conn.close()

ID: vinmec_article_data6215_10, Score: 11.441529838277395
ID: longchau_baiviet_article_data30557_3, Score: 11.06425292496572
ID: vinmec_article_data17055_5, Score: 10.69457584952342
ID: longchau_baiviet_article_data18332_3, Score: 10.544272380072682
ID: vinmec_article_data29262_2, Score: 10.221624573949864
ID: longchau_baiviet_article_data21439_2, Score: 10.18636975340763
ID: longchau_mevabe_article_data2464_2, Score: 9.983057711664141
ID: longchau_baiviet_article_data211_2, Score: 9.851965977218667
ID: longchau_baiviet_article_data12729_0, Score: 9.846674404571841
ID: longchau_baiviet_article_data36220_1, Score: 9.724272460869829

📰 Bài viết liên quan (ID: vinmec_article_data6215_10):
11. Đậu đen Trong tất cả các loại đậu thì đậu đen được coi là một trong những loại đậu lành mạnh nhất. Đậu đen còn được gọi là đậu rùa với hàm lượng chất chống oxy hóa cao có tác dụng giảm nguy cơ ung thư vú và các loại ung thư khác. Đậu đen cũng là một nguồn mạnh mẽ của ergothioneine, một axit amin bảo 

In [ ]:
from llama_cpp import Llama

llm = Llama(model_path="local_model/Vi-Qwen2-1.5B-RAG.Q4_K_M.gguf", n_ctx=8192)


llama_model_loader: loaded meta data with 41 key-value pairs and 338 tensors from local_model/Vi-Qwen2-1.5B-RAG.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Vi Qwen2 1.5B RAG
llama_model_loader: - kv   3:                       general.organization str              = AITeamVN
llama_model_loader: - kv   4:                           general.finetune str              = RAG
llama_model_loader: - kv   5:                           general.basename str              = Vi-Qwen2
llama_model_loader: - kv   6:                         general.size_label str              = 1.5B
llama_model_loader: - kv   7:                     

In [15]:
context = '''- Ngữ cảnh 1: Thái Bình hướng đến là trung tâm công nghiệp, năng lượng của vùng Với tiềm năng sẵn có, quy hoạch tỉnh Thái Bình thời kỳ 2021-2030, tầm nhìn đến năm 2050 xác định tỉnh sẽ phát triển công nghiệp theo hướng hiện đại, bền vững dựa trên nghiên cứu phát triển điện gió, điện khí, cân bằng lượng phát thải. Sáng 5/3, UBND tỉnh Thái Bình tổ chức Hội nghị công bố quy hoạch của tỉnh thời kỳ 2021-2030, tầm nhìn đến năm 2050 và xúc tiến đầu tư tỉnh Thái Bình. Phát biểu tại hội nghị, Phó Chủ tịch Thường trực UBND tỉnh Nguyễn Quang Hưng cho biết: Mục tiêu của quy hoạch là đến năm 2030, Thái Bình trở thành địa phương thuộc nhóm phát triển khá và là một trong những trung tâm phát triển công nghiệp của vùng Đồng bằng sông Hồng, có cơ cấu kinh tế hiện đại với công nghiệp là động lực chủ yếu cho tăng trưởng để Thái Bình phát triển nhanh, toàn diện và bền vững. Đến năm 2050, Thái Bình là tỉnh phát triển của vùng Đồng bằng sông Hồng, tăng trưởng kinh tế dựa trên nền tảng khoa học công nghệ, đổi mới sáng tạo và các ngành kinh tế trụ cột có sức cạnh tranh cao. Quy hoạch tỉnh đã xác định 4 trụ cột tăng trưởng, 3 khâu đột phá, 4 không gian kinh tế - xã hội, 3 hành lang kinh tế, định hướng phát triển các ngành và lĩnh vực và 6 nhiệm vụ trọng tâm. Quy hoạch tỉnh cũng có nhiều điểm mới, đột phá như mở ra không gian phát triển mới thông qua hoạt động “lấn biển”, tạo quỹ đất cho các hoạt động chức năng, hình thành không gian công nghiệp - đô thị - dịch vụ. Về hạ tầng giao thông, Thái Bình sẽ đầu tư 3 tuyến cao tốc là cao tốc Ninh Bình - Hải Phòng (CT.08), đường vành đai 5 - Hà Nội (CT.39) và tuyến CT.16 kết nối Khu kinh tế với thành phố Thái Bình và vùng kinh tế phía Tây Bắc Thủ đô. Tỉnh cũng sẽ đầu tư 101km đường sắt, khổ đường dự kiến rộng 1.435 mm và sân bay chuyên dụng nằm ở ven biển Thái Bình. Về phát triển kinh tế, quy hoạch tỉnh Thái Bình xác định tỉnh sẽ phát triển công nghiệp theo hướng hiện đại, công nghệ tiên tiến, giá trị gia tăng cao, tham gia sâu, toàn diện vào mạng lưới sản xuất, chuỗi giá trị toàn cầu, phát huy các tiềm năng, thế mạnh để đưa Thái Bình trở thành một trong những trung tâm phát triển công nghiệp, năng lượng của vùng Đồng bằng sông Hồng. Tỉnh khuyến khích đầu tư phát triển các ngành có thế mạnh và có thể tạo đột phá như năng lượng, cơ khí chế biến, chế tạo, công nghiệp công nghệ cao, điện - điện tử, chế biến sản phẩm nông, lâm nghiệp và thủy sản… Đồng thời, tập trung nghiên cứu phát triển điện gió, điện khí để tạo nguồn điện sạch và cân bằng lượng phát thải, nghiên cứu đầu tư xây dựng nhà máy chế biến Condensate, chuẩn bị mọi điều kiện để xây dựng và đưa vào vận hành Nhà máy nhiệt điện LNG Thái Bình. Về nông nghiệp, tỉnh Thái Bình vẫn xác định đây là \"trụ cột quan trọng\" trong phát triển kinh tế của tỉnh, góp phần bảo đảm an ninh lương thực quốc gia, hướng tới trở thành trung tâm sản xuất nông nghiệp hàng đầu của Đồng bằng sông Hồng. Phát biểu tại hội nghị, Phó Thủ tướng Chính phủ Trần Lưu Quang đánh giá Thái Bình có 4 tiềm năng, lợi thế lớn để có thể có sự bứt phá trong thời gian tới như vị trí địa lý và tiếp cận đất đai thuận lợi; từng là địa phương đi đầu trong xây dựng nông thôn mới bài bản và nghiêm túc, nhận được sự quan tâm của nhiều thế hệ lãnh đạo Đảng, Nhà nước và có nhiều doanh nhân người Thái Bình và luôn hướng về quê hương; có sự đoàn kết, thống nhất, trước hết là trong tập thể lãnh đạo. Về vị trí địa lý và tiếp cận đất đai, Phó Thủ tướng cho rằng trong tương lai, khi Luật Đất đai có hiệu lực, Thái Bình sẽ có nhiều điều kiện lấn biển để triển khai các dự án khu đô thị, khu công nghiệp thân thiện với môi trường. Đối với nông nghiệp, Phó Thủ tướng nhấn mạnh về lâu dài Thái Bình có thể ghi điểm từ phát triển công nghiệp nhưng trước mắt, đặc biệt trong lúc khó khăn thì nông nghiệp vẫn là nền tảng rất quý giá. Mặt khác, ứng dụng của công nghệ cao trong sản xuất nông nghiệp sẽ rút ngắn thời gian làm đồng của người nông dân, tạo điều kiện để Thái Bình huy động nguồn nhân lực trong nông nghiệp sang phát triển các ngành công nghiệp và dịch vụ, một lợi thế mà không phải địa phương nào cũng có được như Thái Bình. Bên cạnh những lợi thế trên, lãnh đạo Chính phủ chỉ ra một số khó khăn mà tỉnh phải đối mặt như Thái Bình đã sử dụng hết 1.600 ha chỉ tiêu đất công nghiệp trong giai đoạn này, đòi hỏi phải có phương án giải quyết thấu đáo trong thời gian tới để tỉnh tiếp tục phát triển công nghiệp. Đồng thời, Thái Bình cũng phải cạnh tranh với những địa phương như Hải Phòng, Quảng Ninh trong thu hút FDI trong khi phát triển cơ sở hạ tầng chưa theo kịp mong muốn. Do vậy, khi triển khai quy hoạch tỉnh, Phó Thủ tướng nhắn nhủ tới địa phương 8 chữ: Tuân thủ, linh hoạt, đồng bộ và thấu hiểu. Đồng thời, tỉnh cũng phải \"linh hoạt\" trong tổ chức thực hiện, trong trường hợp cá biệt cụ thể, điều chỉnh mục tiêu cho phù hợp. Sáng cùng ngày, Phó Thủ tướng Trần Lưu Quang đã dự Lễ khởi công dự án Nhà máy Pegavision Việt Nam tại khu công nghiệp Liên Hà Thái, huyện Thái Thụy, tỉnh Thái Bình 

- Ngữ cảnh 2: Bình Định được định hướng là trung tâm khoa học, công nghệ đổi mới sáng tạo Tỉnh Bình Định được định hướng phát triển ngành công nghiệp phát triển theo hướng hiện đại, quy mô lớn, trở thành một trong những trung tâm công nghiệp chế biến chế tạo và công nghệ cao của vùng Bắc Trung Bộ và duyên hải Trung Bộ. Theo Quy hoạch tỉnh Bình Định thời kỳ 2021 - 2030, tầm nhìn đến năm 2050 vừa được Thủ tướng Chính phủ phê duyệt, tỉnh Bình Định được định hướng phát triển ngành công nghiệp phát triển theo hướng hiện đại, quy mô lớn, trở thành một trong những trung tâm công nghiệp chế biến chế tạo và công nghệ cao của vùng Bắc Trung Bộ và duyên hải Trung Bộ. Ngành công nghiệp tăng trưởng nhanh, bền vững, hướng tới tăng trưởng xanh, kinh tế tuần hoàn là trụ cột để phát triển và chuyển dịch cơ cấu kinh tế của tỉnh. Ngành chế biến, chế tạo công nghệ cao (dịch chuyển ngành công nghiệp chế biến, chế tạo sang lĩnh vực sản xuất có giá trị gia tăng cao như: chế biến sâu nông - thủy - hải sản, linh kiện điện tử, bán dẫn, dược phẩm), công nghệ thông tin, trí tuệ nhân tạo trở thành một trong những lĩnh vực đột phá, góp phần đưa tỉnh Bình Định trở thành một trung tâm khoa học, công nghệ đổi mới sáng tạo của vùng và cả nước. Quy hoạch tỉnh Bình Định thời kỳ 2021 - 2030, tầm nhìn đến năm 2050 đặt ra yêu cầu tỉnh này phải chú trọng thu hút đầu tư phát triển năng lượng tái tạo, năng lượng sạch như điện gió ven bờ, điện gió ngoài khơi, điện mặt trời, điện sinh khối và nguồn năng lượng mới (hydrogen/amoniac xanh…); các dự án sản xuất thép quy mô lớn, đóng tàu, sản xuất thiết bị phụ trợ điện gió có công nghệ tiên tiến để nâng cấp xây dựng hạ tầng kỹ thuật sản xuất, thúc đẩy chuyển dịch kinh tế. Quy hoạch tỉnh Bình Định thời kỳ 2021 - 2030, tầm nhìn đến năm 2050 cũng đặt ra mục tiêu đến năm 2030, Bình Định trở thành tỉnh phát triển thuộc nhóm dẫn đầu vùng Bắc Trung Bộ và duyên hải Trung Bộ, là trung tâm công nghiệp chế biến, chế tạo, dịch vụ, du lịch và văn hóa phía Nam của vùng; trung tâm lớn của cả nước về phát triển kinh tế biển; trọng điểm du lịch quốc gia và quốc tế với hệ thống kết cấu hạ tầng kinh tế đồng bộ, hiện đại; kinh tế của tỉnh phát triển nhanh, bền vững và xanh dựa trên các trụ cột tăng trưởng công nghiệp, dịch vụ du lịch, cảng biển - logistics; nông nghiệp ứng dụng công nghệ cao; đô thị hóa; thực hiện thành công các mục tiêu chuyển đổi số, đổi mới sáng tạo, cải thiện mạnh mẽ môi trường đầu tư kinh doanh, trở thành điểm đến đầu tư hấp dẫn của các doanh nghiệp lớn trong và ngoài nước; chỉ số năng lực cạnh tranh cấp tỉnh thuộc nhóm cao của cả nước; kết cấu hạ tầng kinh tế - xã hội đồng bộ, hiện đại, hệ thống đô thị phát triển theo hướng đô thị thông minh, kết nối thuận tiện với các trung tâm kinh tế của vùng, cả nước và quốc tế. 
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
- Ngữ cảnh 3: . Chủ tịch UBND tỉnh Quảng Ninh cho biết, tỉnh đặt mục tiêu hướng đến năm 2030 trở thành một tỉnh tiêu biểu của cả nước về mọi mặt; tỉnh kiểu mẫu giàu đẹp, văn minh, hiện đại, nâng cao đời sống mọi mặt của nhân dân; cực tăng trưởng của khu vực phía Bắc, một trong những trung tâm phát triển năng động, toàn diện; trung tâm du lịch quốc tế, trung tâm kinh tế biển, cửa ngõ của Vùng kinh tế trọng điểm Bắc Bộ và cả nước. Để đạt được những mục tiêu trên, tỉnh Quảng Ninh xác định sự đóng góp, quan tâm của cộng đồng doanh nghiệp, nhất là các doanh nghiệp hàng đầu Việt Nam “các sếu đầu đàn” là một trong những yếu tố then chốt quyết định. Do vậy, tỉnh Quảng Ninh rất mong nhận được sự quan tâm, nghiên cứu đầu tư hợp tác của các Doanh nghiệp hàng đầu Việt Nam trong thời gian tới, nhất là trong việc đầu tư các dự án có hàm lượng công nghệ cao, công nghệ tiên tiến, quản trị hiện đại, giá trị gia tăng cao, có tác động lan tỏa. Tỉnh Quảng Ninh cam kết tạo điều kiện thuận lợi nhất cho doanh nghiệp phát triển hơn nữa khi đầu tư kinh doanh trên địa bàn tỉnh; cam kết đồng hành, lắng nghe tiếng nói của cộng đồng doanh nghiệp, các nhà đầu tư; cùng trăn trở, trách nhiệm, giải quyết thấu đáo, vào cuộc thực chất, hiệu quả đối với từng khó khăn, vướng mắc với mục tiêu tăng cường niềm tin và nâng cao sự hài lòng của cá nhân, tổ chức, doanh nghiệp là thước đo đánh giá chất lượng phục vụ, chất lượng điều hành của cơ quan hành chính nhà nước từ cấp tỉnh đến các cấp cơ sở. Tỉnh Quảng Ninh sẽ triển khai một cách hiệu quả nhất Quy hoạch tỉnh, quy hoạch các địa phương; chuẩn bị mặt bằng sạch; quỹ đất tái định cư; đảm bảo nguồn cung điện, nước và viễn thông theo yêu cầu; xây dựng các chính sách đột phá để thu hút nguồn nhân lực chất lượng cao thông qua việc cải thiện nhà ở và cơ sở văn hóa để đảm bảo đời sống vật chất và tinh thần cho người lao động. Đồng thời bảo vệ quyền và lợi ích chính đáng, hợp pháp của nhà đầu tư, doanh nghiệp vào tỉnh; bảo đảm hài hòa lợi ích giữa nhà nước, nhà đầu tư, người lao động và người dân.'''

question = '''Theo quy hoạch tỉnh Tây Ninh thời kỳ 2021 - 2030, tầm nhìn đến năm 2050, tỉnh sẽ tập trung phát triển công nghiệp và dịch vụ du lịch. Trong bối cảnh cạnh tranh với các tỉnh lân cận, yếu tố nào được coi là quan trọng nhất để Tây Ninh có thể thu hút đầu tư và phát triển nhanh, toàn diện và bền vững?'''


'''Trả lời:
Trong ngữ cảnh được cung cấp, không có thông tin về quy hoạch tỉnh Tây Ninh thời kỳ 2021 - 2030, tầm nhìn đến năm 2050. Do đó, không có câu trả lời chính xác dựa trên các thông tin đã cung cấp. Nếu bạn có thêm thông tin về quy hoạch tỉnh Tây Ninh, vui lòng cung cấp để tôi có thể trả lời câu hỏi một cách chính xác và đầy đủ.
'''


system_prompt = "Bạn là một trợ lí Tiếng Việt nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể."
template = f"""Chú ý các yêu cầu sau:
- Câu trả lời phải chính xác và đầy đủ nếu ngữ cảnh có câu trả lời. 
- Chỉ sử dụng các thông tin có trong ngữ cảnh được cung cấp.
- Chỉ cần từ chối trả lời và không suy luận gì thêm nếu ngữ cảnh không có câu trả lời.

Hãy trả lời câu hỏi dựa trên ngữ cảnh:

### Ngữ cảnh :
{context}

### Câu hỏi :
{question}

### Trả lời :
"""

full_prompt = f"[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{template} [/INST]"

In [16]:
buffer = ''
answer = ""
for chunk in llm(full_prompt, max_tokens=1024, stream=True):
    text = chunk["choices"][0]["text"]
    buffer+=text
    # if 'Llama.generate: 4392 prefix-match hit, remaining 1 prompt tokens to eval' in buffer:
    #     continue
    if '[/INST]' in buffer:
        break
    answer+=text

Llama.generate: 131 prefix-match hit, remaining 2758 prompt tokens to eval


In [17]:
print(answer.split('[/INST')[0])

 Không có thông tin về quy hoạch tỉnh Tây Ninh thời kỳ 2021 - 2030, tầm nhìn đến năm 2050 trong ngữ cảnh được cung cấp. Do đó, không thể trả lời chính xác câu hỏi này dựa trên các thông tin đã cho. Nếu bạn có thêm thông tin về Tây Ninh, tôi sẽ rất vui được giúp đỡ. 


In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
path = 'AITeamVN/Vi-Qwen2-1.5B-RAG'
model = AutoModelForCausalLM.from_pretrained(
    path,
    torch_dtype=torch.bfloat16, 
    device_map="auto",
    use_cache=True
)
tokenizer = AutoTokenizer.from_pretrained(path)

system_prompt = "Bạn là một trợ lí Tiếng Việt nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể."
template = '''Chú ý các yêu cầu sau:
- Câu trả lời phải chính xác và đầy đủ nếu ngữ cảnh có câu trả lời. 
- Chỉ sử dụng các thông tin có trong ngữ cảnh được cung cấp.
- Chỉ cần từ chối trả lời và không suy luận gì thêm nếu ngữ cảnh không có câu trả lời.
Hãy trả lời câu hỏi dựa trên ngữ cảnh:
### Ngữ cảnh :
{context}

### Câu hỏi :
{question}

### Trả lời :'''

# Ví dụ
context = '''Thuốc Insuact 10 trị bệnh gì? Thuốc Insuact 10mg có thành phần chính là Atorvastatin. Thuốc Insuact 10 có tác dụng làm giảm cholesterol và triglycerid trong máu ở bệnh nhân tăng cholesterol máu nguyên phát, rối loạn lipid máu hỗn hợp. 1. Thuốc Insuact 10 trị bệnh gì? Thuốc Insuact 10 thuộc nhóm thuốc điều trị rối loạn lipid máu, có thành phần chính là Atorvastatin 10mg. Atorvastatin có tác dụng làm giảm cholesterol, ức chế enzym tạo cholesterol ở gan. Atorvastatin làm giảm cholesterol chung bao gồm cholesterol LDL , triglycerid trong máu. Thuốc Insuact 10mg được bào chế dưới dạng viên nén bao phim, được chỉ định dùng trong những trường hợp sau: Ðiều trị hỗ trợ tăng cholesterol máu nguyên phát và rối loạn lipid máu hỗn hợp trên bệnh nhân đang áp dụng chế độ ăn kiêng để làm giảm cholesterol toàn phần , cholesterol LDL , apolipoprotein B, triglycerid và tăng cholesterol HDL . Insuact 10 cũng được dùng để điều trị rối loạn betalipoprotein trong máu nguyên phát. Ðiều trị hỗ trợ tăng cholesterol trong máu có tính gia đình đồng hợp tử trên bệnh nhân đang áp dụng các biện pháp làm giảm lipid khác để làm giảm cholesterol toàn phần và cholesterol LDL. 2. Liều dùng và cách dùng thuốc Insuact 10 Cách dùng thuốc Insuact 10: Thuốc được dùng theo đường uống, uống khi bụng đói hoặc no đều được, có thể uống vào bất cứ lúc nào trong ngày. Liều dùng thuốc Insuact 10mg khởi đầu là 10mg/lần/ngày, tối đa là 80mg/lần/ngày. Liều dùng thuốc Insuact 10 tùy vào mục đích điều trị cụ thể như sau: Tăng cholesterol máu nguyên phát và rối loạn lipid máu phối hợp: 10mg/lần/ngày, sau 2 - 4 tuần sẽ thấy hiệu quả của thuốc. Thuốc cần được được sử dụng duy trì trong thời gian dài để có hiệu quả. Tăng cholesterol trong máu có tính gia đình đồng hợp tử: Liều thường dùng là thuốc Insuact 10mg /lần/ngày và tối đa là 80mg/lần/ngày. Rối loạn lipid máu nghiêm trọng ở trẻ từ 10 - 17 tuổi: 10mg/lần/ngày, sau đó tăng lên 20mg/lần/ngày tùy vào cơ địa, tiến triển bệnh và khả năng dung nạp thuốc của người bệnh. Thời gian điều chỉnh liều thuốc tối thiểu là 4 tuần. 3. Tác dụng phụ của thuốc Insuact 10mg Thuốc Insuact 10 có thể gây một số tác dụng phụ không mong muốn với tần suất như sau: Thường gặp: Viêm mũi - họng, phản ứng dị ứng, tăng đường huyết, nhức đầu, đau thanh quản, chảy máu cam , đau cơ, co thắt cơ, đau khớp, sưng khớp, đau các chi, đau lưng, xét nghiệm gan bất thường, tăng creatine kinase trong máu, buồn nôn, khó tiêu, đầy hơi, táo bón, tiêu chảy. Ít gặp: Insuact 10 ít gây hạ đường huyết, tăng cân, chán ăn, mất ngủ, gặp ác mộng, choáng váng, dị cảm, mất trí nhớ, giảm cảm giác, loạn vị giác , nôn, đau bụng, ợ hơi, viêm tụy, viêm gan, nổi mày đay , phát ban, ngứa, rụng tóc, đau cổ, mỏi cơ, mệt mỏi, suy nhược, đau ngực, phù ngoại biên, sốt, xuất hiện bạch cầu trong nước tiểu, nhìn mờ, ù tai. Hiếm gặp: Insuact 10 hiếm khi làm giảm tiểu cầu, bệnh lý thần kinh ngoại biên, hoa mắt, ứ mật, phù thần kinh, nổi hồng ban, hội chứng hoại tử da nhiễm độc , hội chứng Stevens-Johnson , bệnh cơ, viêm cơ, tiêu cơ vân, bệnh gân, đôi khi nghiêm trọng hơn có thể đứt gân. Rất hiếm gặp: Insuact 10 rất hiếm khi gây sốc phản vệ , mất thính giác , suy gan , hội chứng to vú ở nam giới. Không rõ tần suất: Hoại tử cơ tự miễn trung gian. 4. Một số lưu ý khi dùng thuốc Insuact 10mg Không dùng thuốc Insuact với người bị quá mẫn với thành phần của thuốc, người có bệnh gan hoạt động hoặc tăng transaminase huyết thanh vô căn kéo dài, phụ nữ đang mang thai hoặc nuôi con cho bú, phụ nữ đang có ý định mang thai. Thuốc Insuact 10mg chỉ được dùng ở bệnh nhân có nguy cơ xơ vữa mạch máu cao do tăng cholesterol trong máu và phải kết hợp với chế độ ăn kiêng ít chất béo bão hòa , ít cholesterol và người bệnh đang áp dụng các biện pháp điều trị không dùng thuốc khác. Trước khi điều trị với Insuact 10 , người bệnh cần được loại trừ các nguyên nhân thứ phát gây tăng cholesterol bao gồm suy tuyến giáp , tiểu đường khó kiểm soát, hội chứng thận hư, nghiện rượu, bệnh gan tắc nghẽn, rối loạn protein trong máu, .... Ngoài ra, người bệnh cũng cần được kiểm tra, đo lường nồng độ lipid máu. Trước khi điều trị với thuốc Insuact 10mg , cần kiểm tra chức năng gan và trong quá trình dùng thuốc, người bệnh cần theo dõi, kiểm tra chức năng gan thường xuyên. Người có tiền sử mắc bệnh gan do rượu, bị nghiện rượu cần thận trọng khi dùng Insuact 10 . Trước khi dùng thuốc, người bệnh cần được cảnh báo nguy cơ gặp phải các vấn đề về cơ như căng cơ , đau cơ, yếu cơ. Thuốc Insuact 10mg có thể tương tác với các thuốc: Tăng nguy cơ tổn thương cơ (đặc biệt cơ vân) khi dùng đồng thời với thuốc điều trị viêm gan C và HIV, hạ cholesterol máu nhóm fibrat khác, thuốc ức chế miễn dịch; tăng thời gian chảy máu hoặc thời gian prothrombin khi dùng phối hợp với thuốc chống đông, indandione; tăng nhẹ nồng độ digoxin trong huyết thanh khi dùng phối hợp với Digoxin'''
question = '''Insuact 10mg được chỉ định dùng trong những trường hợp nào và liều dùng cụ thể ra sao?'''
conversation = [{"role": "system", "content": system_prompt }]
conversation.append({"role": "user", "content": template.format(context = context, question = question)})
text = tokenizer.apply_chat_template(
    conversation,
    tokenize=False,
    add_generation_prompt=True)
model_inputs = tokenizer(text,return_tensors="pt").to(model.device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=2048,
    temperature = 0.1,
    #top_p=0.95,
    #top_k=40,
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

'''Trả lời:
Dựa trên ngữ cảnh cung cấp, Insuact 10mg được chỉ định dùng trong các trường hợp sau:

1. **Điều trị hỗ trợ tăng cholesterol máu nguyên phát và rối loạn lipid máu hỗn hợp**:
   - **Liều dùng**: 10mg/lần/ngày.
   - **Hiệu quả**: Sau 2 - 4 tuần sử dụng, bệnh nhân sẽ thấy hiệu quả của thuốc.
   - **Lưu ý**: Thuốc cần được sử dụng duy trì trong thời gian dài để có hiệu quả.

2. **Điều trị hỗ trợ tăng cholesterol trong máu có tính gia đình đồng hợp tử**:
   - **Liều dùng**: 10mg/lần/ngày (liều thường dùng) và tối đa 80mg/lần/ngày.
   - **Lưu ý**: Liều lượng có thể được điều chỉnh tùy theo cơ địa, tiến triển bệnh và khả năng dung nạp thuốc của người bệnh.

3. **Rối loạn lipid máu nghiêm trọng ở trẻ từ 10 - 17 tuổi**:
   - **Liều dùng**: 10mg/lần/ngày, sau đó có thể tăng lên 20mg/lần/ngày tùy theo tình trạng bệnh.
   - **Thời gian điều chỉnh liều**: Thời gian điều chỉnh liều tối thiểu là 4 tuần.

**Lưu ý chung**:
- Thuốc được dùng theo đường uống, uống khi bụng đói hoặc no đều được, có thể uống vào bất cứ lúc nào trong ngày.
- Không dùng thuốc Insuact 10mg với người bị quá mẫn với thành phần của thuốc, người có bệnh gan hoạt động hoặc tăng transaminase huyết thanh vô căn kéo dài, phụ nữ đang mang thai hoặc nuôi con cho bú, phụ nữ đang có ý định mang thai.
- Cần kiểm tra chức năng gan trước và trong quá trình điều trị.

'''


ModuleNotFoundError: No module named 'torch'